# COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases in Asia

- author: Hargun Oberoi
- image: images/covid-overview.png
- permalink: /covid-overview-asia/
- hide: true
- badges: false

In [1]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [2]:
#hide

# FETCH
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

In [3]:
#hide
paths['mapping']

'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/mapping_countries.csv'

In [4]:
#hide
def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

In [5]:
#hide
mapping = get_mappings(paths['mapping'])

In [6]:
#hide
def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

In [7]:
#hide
def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

In [8]:
#hide
def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [9]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

In [10]:
#hide
dt_today =dt_cols[LAST_DATE_I]
# dt_5day = dt_cols[LAST_DATE_I - 5]
dt_yday = dt_cols[LAST_DATE_I - 1]

In [11]:
#hide
dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
# dfp_cases = dft_cases.groupby(COL_REGION)[dt_5day].sum()
# dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5day].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_yday].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_yday].sum()

In [12]:
#hide
df_table = (pd.DataFrame(dict(Cases = dfc_cases, Deaths = dfc_deaths, PCases = dfp_cases, PDeaths = dfp_deaths))
            .sort_values(by = ['Cases','Deaths'], ascending = [False, False])
            .reset_index())

In [13]:
#hide
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)
    #Clip ça veut dire, les chiffres negatif sont interdit
df_table['Fatality Rate'] = (100* df_table['Deaths']/ df_table['Cases']).round(2)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table = df_table[df_table.Continent == 'Asia']
df_table.head()

,Country/Region,Cases,Deaths,PCases,PDeaths,Cases (+),Deaths (+),Fatality Rate,Continent
1,India,10323965,149435,10286709,148994,37256,441,1.45,Asia
6,Turkey,2232035,21295,2220855,21093,11180,202,0.95,Asia
14,Iran,1237474,55438,1231429,55337,6045,101,4.48,Asia
19,Indonesia,758473,22555,751270,22329,7203,226,2.97,Asia
24,Iraq,597033,12829,596193,12824,840,5,2.15,Asia


In [14]:
#hide
#delete problematic countries from table
df_table = df_table[~df_table['Country/Region'].isin(['Cape Verde', 'Cruise Ship', 'Kosovo'])]

In [15]:
#hide
metrics = [df_table.columns[index] for index in [1,2,5,6]]
# s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_turkey = df_table[df_table['Country/Region'].eq('Turkey')][metrics].sum().add_prefix('Turkey ')
s_iran = df_table[df_table['Country/Region'].eq('Iran')][metrics].sum().add_prefix('Iran ')
# s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
# s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
s_ind = df_table[df_table['Country/Region'].eq('India')][metrics].sum().add_prefix('India ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_yday)}
summary = {**summary, **df_table[metrics].sum(), **s_ind, **s_iran, **s_turkey}
# summary = {**summary, **df_table[metrics].sum(), **s_ind, **s_eu, **s_us}

In [16]:
#hide
dft_ct_new_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum().diff(axis=1).fillna(0).astype(int)

In [17]:
#hide
def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()
#Vraiment, je sais pas, c'est quoi

In [18]:
#hide
KPI_CASE = 'Asia'
kpis_info = [
    {'title': 'India', 'prefix': 'India'},
    {'title': 'Turkey', 'prefix': 'Turkey'},
    {'title': 'Iran', 'prefix': 'Iran'}]

In [19]:
#hide_input
template = Template(get_template('https://raw.githubusercontent.com/hargun3045/dashboards/master/overview.tpl'))
# with open('overview.tpl','r') as f:
    # temp = f.read()
# template = Template(template)
html = template.render(
    D=summary, table=df_table.head(10),
    KPI_CASE = KPI_CASE,
    KPIS_INFO = kpis_info,# REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-21]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/)